<a href="https://colab.research.google.com/github/hdyoon/AI_lecture/blob/master/simple_perceptron_AND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

simple perceptron 구현
=================


본 장에서는 단층 퍼셉트론을 파이썬으로 구현하여 보도록 하겠습니다. 먼저 우리가 구현할 AND 논리 연산은 다음과 같습니다.

| $x_1$ 	| $x_2$ 	| $x_1 \cap x_2$ 	| $x_1 \cup x_2$ 	| $x_1 \oplus x_2$ 	|
|:-----:	|:-----:	|:--------------:	|:--------------:	|:----------------:	|
|     0 	|     0 	|              -1 	|             -1 	|               -1 	|
|     0 	|     1 	|              -1 	|              1 	|                1 	|
|     1 	|     0 	|              -1	|              1 	|                1 	|
|     1 	|     1 	|               1	|              1 	|               -1 	|

activation function으로 sign function을 사용하게 됨으로 False일 경우 -1 True일 경우 1을 반환하고 있습니다.

![Activation Functions](https://4.bp.blogspot.com/-e96MNS1bxts/WCsKyopRHkI/AAAAAAAACMU/9CJNMzXpIXUBVV9tD5A2nWaEQZf8nA-LwCLcB/s320/Activation%2BFunction.PNG "Activation Functions")


퍼셉트론 훈련 절차
-----------------

1. $w_0 ~ w_n$의 값을 0.0, 임계값(threshold) 0.0, 에타값은 0.1로 세팅한다.
2. 훈련 데이터의 특성값들에 대한 예측값의 활성 함수 리턴값을 계산한다.
3. 2에서 계산된 값이 이 실제 결과값의 활성 함수 리턴값과 같으면 가중치 업데이트 없이 다음 트레이닝 데이터에 대해 2번 과정으로 넘어간다.
4. 예측값의 활성 함수 리턴값이 실제 결과값의 활성 함수 리턴값과 다르면 델타규칙에 의해 $w_0 ~ w_n$의 가중치를 업데이트 한다.
5. 더 이상 업데이트가 일어나질 않을때까지 2~4를 반복한다.

각 노드 사이의 가중치는 delta rule에 의해서 업데이트 됩니다. 

<center>$\Delta w_j = \eta (y - \hat{y})x_j$</center>

### 클래스 생성 및 변수 초기화
* eta : 학습 속도
* n_iter : 최대 반복 횟수 설정
* threshold : 활성 함수의 임계 기준

~~~python
def __init__(self, eta=0.01, threshold=0.0, n_iter=10):
    self.eta = eta
    self.threshold = threshold
    self.n_iter = n_iter
~~~

### 퍼셉트론 학습
퍼셉트론의 학습을 진행하는 fit함수를 정의해보도록 합시다. 각 노드 사이의 가중치 값은 w 배열에 담기게 됩니다. 코드에서는 w배열크기를 입력값 배열크기에 +1을 하는데, 그 이유는 bias를 위한 공간을 마련하기 위해서입니다. 지금처럼 x값으로 들어오는 입력값이 2개인 경우 [w0, w1, w2] 모양의 배열이 생성되며, 0번째 인덱스의 방이 바로 b값을 위한 공간으로 세팅 됩니다. 
errors 배열은 train 과정을 저장하게 됩니다. 퍼셉트론은 모든 가중치가 더이상 업데이트가 안 일어날때 학습이 끝났다고 볼 수 있습니다. 즉, 더 이상 예측값과 실제값이 차이가 발생하지 않을 때 모델이 완성됩니다.
~~~python
update = self.eta * (target - self.predict(xi))
~~~
부분에서 delta rule을 구현하고 있습니다. 가중치는 실제값과 예측값의 차이에 따라 업데이트 됩니다.

~~~python
def fit(self, X, y):
    self.w_ = np.zeros(1 + X.shape[1])
    self.errors_ = []
    for _ in range(self.n_iter):
        errors = 0
        for xi, target in zip(X, y):
            update = self.eta * (target - self.predict(xi))
            self.w_[1:] += update * xi
            self.w_[0] += update
            errors += int(update != 0.0)
        self.errors_.append(errors)
    return self
~~~

### 예측값 계산

numpy의 matmul함수는 두 행렬의 곱을 반환합니다. 즉, 각 입력값에 대한 가중치의 곱을 모두 더한 값을 계산하게 됩니다.

$\begin{Bmatrix} x_1 & x_2 \end{Bmatrix} \times \begin{Bmatrix} w_1 \\ w_2 \end{Bmatrix} = \begin{Bmatrix} x_1 \times w_1 + x_2 \times w_2 \end{Bmatrix} = \sum_{i=1}^2{x_iw_i}$

predict는 그 해당 값이 임계값을 넘으면 1 아니면 -1을 반환하는 activation function 역할을 합니다.
여기까지하면 Perceptron 클래스의 작성이 완료되었습니다.

~~~python
def net_input(self, X):
    return np.matmul(X, self.w_[1:]) + self.w_[0]
    
def predict(self, X):
    return np.where(self.net_input(X) >= self.threshold, 1, -1)
~~~

### Main
Perceptron 클래스 생성을 완료하고 입력값(X)과 실제출력값(y)을 이용하여 퍼셉트론을 훈련한다.

~~~python
if __name__ == '__main__':
    X = np.array([[0, 0],[0, 1],[1, 0],[1, 1]])
    y = np.array([-1, -1, -1, 1])
    
    ppn = Perceptron(eta=0.1)
    ppn.fit(X, y)
    print(ppn.errors_)
~~~

## 전체 소스

In [28]:
import numpy as np

class Perceptron(object):
    def __init__(self, eta=0.01, threshold=0.0, n_iter=10):
        self.eta = eta
        self.threshold = threshold
        self.n_iter = n_iter
        
    def fit(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        return np.matmul(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        return np.where(self.net_input(X) >= self.threshold, 1, -1)

if __name__ == '__main__':
    X = np.array([[0, 0],[0, 1],[1, 0],[1, 1]])
    y = np.array([-1, -1, -1, 1])
    
    ppn = Perceptron(eta=0.1)
    ppn.fit(X, y)
    print(ppn.errors_)

[2, 3, 3, 0, 0, 0, 0, 0, 0, 0]
